# 1. Run segmentation 

We want to perform segmentation to identify cells using SOPA. Further information can be found at [this link](https://gustaveroussy.github.io/sopa)

## Import packages

In [1]:
import scanpy as sc
import sopa
import spatialdata as sd


/home/sergio/.local/lib/python3.10/site-packages/dask/dataframe/__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(
/home/sergio/.local/lib/python3.10/site-packages/anndata/utils.py:429: FutureWarning: Importing read_text from `anndata` is deprecated. Import anndata.io.read_text instead.
  warnings.warn(msg, FutureWarning)


## Read SpatialData dataset

We read the sdata cropped object we created in ``0.format_xenium_sdata.ipynb``

In [36]:
xenium_path_cropped = "/media/sergio/Discovair_final/mousebrain_prime_crop_quantified.zarr"
sdata = sd.read_zarr(xenium_path_cropped)

Next we will create the image patches needed for segmentation

In [37]:
sopa.make_image_patches(sdata)  # creating overlapping patches

[INFO] (sopa.patches._patches) 6 patches were added to sdata['image_patches']


We will run Cellpose segmentation using the DAPI and 18S layers in this example

In [64]:
sopa.segmentation.cellpose(sdata, ["DAPI", "18S"], diameter=2)  # running cellpose segmentation
sopa.aggregate(sdata, shapes_key="cellpose_boundaries", key_added="table", gene_column="feature_name")

[WARNING] (sopa._settings) Running without parallelization backend can be slow. Consider using a backend, e.g. via `sopa.settings.parallelization_backend = 'dask'`, or `export SOPA_PARALLELIZATION_BACKEND=dask`.
100%|██████████| 6/6 [2:02:34<00:00, 1225.80s/it]  
[INFO] (sopa.segmentation._stainings) Found 1750 total cells
Resolving conflicts: 100%|██████████| 24/24 [00:00<00:00, 3133.29it/s]
[INFO] (sopa.segmentation._stainings) Added 1738 cell boundaries in sdata['cellpose_boundaries']
[INFO] (sopa.aggregation.transcripts) Aggregating transcripts over 1738 cells


[########################################] | 100% Completed | 9.75 ss


[INFO] (sopa.aggregation.channels) Aggregating channels intensity over 1738 cells with mode='average'


[########################################] | 100% Completed | 505.44 ms


[INFO] (sopa.aggregation.aggregation) Filtering 0 cells


We plot the location of cells 

In [ ]:
sc.pl.spatial(sdata.tables["table"], spot_size=10, palette="viridis")

Finally, we save the spatialdata object as zarr

In [ ]:
xenium_path_cropped = "/media/sergio/Discovair_final/mousebrain_prime_crop_points2regions.zarr"
sdata.write(xenium_path_cropped, overwrite=True)